# This code gets the keyword frequencies and topics mentioned!

In [ ]:
import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
cleaned_data = pd.read_csv("../Output/20240419_CN_NDC_sentences_qc.csv")
cleaned_data

In [ ]:
keywords_df = pd.read_csv("../Data/keywords_for_marine_policy.csv")
keywords_df

In [ ]:
stemmer = PorterStemmer()

stemmed_keywords = ([stemmer.stem(keyword) for keyword in keywords_df["Keyword"]])
keywords = list(keywords_df['Keyword'])
keywords 

In [ ]:
len(stemmed_keywords)

In [ ]:
len(set(keywords))

In [ ]:
stemmed_keywords

In [ ]:
keyword_topics = {}
for index in keywords_df.index:
    if not keyword_topics.get(keywords_df.loc[index, "Keyword"]):
        keyword_topics[keywords_df.loc[index, "Keyword"]] = [keywords_df.loc[index, "Topic"]]
    else:
        keyword_topics[keywords_df.loc[index, "Keyword"]].append(keywords_df.loc[index, "Topic"])
keyword_topics

In [ ]:
unique_topics = set(keywords_df['Topic'])
unique_topics

In [ ]:
def count_keywords(text, keyword_set):
    text = text.lower()
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    text = ''.join(char for char in text if char not in punctuations)
    
    counts = {}
    for index in range(len(keyword_set)):
        counts[keywords[index]] = text.count(keyword_set[index])
    return counts

In [ ]:
keyword_counts = cleaned_data.apply(lambda row: count_keywords(row['text_content'], stemmed_keywords), axis=1, result_type='expand')
cleaned_data = pd.concat([cleaned_data, keyword_counts], axis=1)
cleaned_data

In [ ]:
def count_topic(text, topics):
    text = text.lower()
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    text = ''.join(char for char in text if char not in punctuations)

    topic_counts_local = {topic: 0 for topic in topics}
    for index in range(len(stemmed_keywords)):
        freq = text.count(stemmed_keywords[index])
        for topic in keyword_topics[keywords[index]]:
            topic_counts_local[topic] += freq
    return topic_counts_local

In [ ]:
topic_keyword_counts = cleaned_data.loc[:, 'text_content'].apply(lambda x: count_topic(x, unique_topics)).apply(pd.Series)
cleaned_data = pd.concat([cleaned_data, topic_keyword_counts], axis=1)
cleaned_data


In [ ]:
cleaned_data.to_csv("../Output/topic_keyword_freq_NDC_sentences_CN.csv", index=False)